# cs7324 Lab 6 - Sequential Networks

#### Chip Henderson - 48996654 


## Preparation

In this lab, I'll be using a collection of Tweets related to the coronavirus pandemic. My intent is to use Convolutional Neural Network and Transformer models to analyze sentiment.

Source: https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification

#### Data Import and Tokenization

My dataset is already split into training and testing categories. Since I will need to split out my y values as well, I'm going to combine the two.

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import sequence

# I'm using a latin enoding because there are some special characters
X_train = pd.read_csv(r'C:\Users\Chip\source\repos\cs7324_code\Data_Sources\Coronavirus_tweets\Corona_NLP_train.csv',encoding='iso-8859-1')
X_test = pd.read_csv(r'C:\Users\Chip\source\repos\cs7324_code\Data_Sources\Coronavirus_tweets\Corona_NLP_test.csv',encoding='iso-8859-1')

X = pd.concat([X_train, X_test])

X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44955 entries, 0 to 3797
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       44955 non-null  int64 
 1   ScreenName     44955 non-null  int64 
 2   Location       35531 non-null  object
 3   TweetAt        44955 non-null  object
 4   OriginalTweet  44955 non-null  object
 5   Sentiment      44955 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.4+ MB


Because of the memory footprint and compute times for some of these networks, I'm going to make my dataset as simple as possible. Therefore I'll be dropping everything except for the Original Tweet and Sentiment columns.

In [3]:
features_to_keep = ['OriginalTweet', 'Sentiment']
features_to_drop = [feature for feature in X.columns if feature not in features_to_keep]

X_reduced = X.drop(features_to_drop,axis=1)

X_reduced.head(10)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,As news of the regionÂs first confirmed COVID...,Positive
6,Cashier at grocery store was sharing his insig...,Positive
7,Was at the supermarket today. Didn't buy toile...,Neutral
8,Due to COVID-19 our retail store and classroom...,Positive
9,"For corona prevention,we should stop to buy th...",Negative
